# ETL

In [210]:
# Se importan las librerias usadas en el código
import pandas as pd
import numpy as np

In [211]:
data=pd.read_csv(r"D:\Programacion\DataScience_Henry\Proyecto_Individual1\Dataset\movies_dataset.csv")

C:\Users\lasso\AppData\Local\Temp\ipykernel_16552\857184425.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(r"D:\Programacion\DataScience_Henry\Proyecto_Individual1\Dataset\movies_dataset.csv")


In [212]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [213]:
data.drop(columns=["video","imdb_id","adult","original_title","poster_path" , "homepage"],inplace=True)

In [214]:
data["budget"].value_counts()

0           36573
5000000       286
10000000      259
20000000      243
2000000       242
            ...  
923             1
72500000        1
2160000         1
4439832         1
1254040         1
Name: budget, Length: 1226, dtype: int64

In [215]:
data["revenue"].value_counts()

0.0           38052
12000000.0       20
11000000.0       19
10000000.0       19
2000000.0        18
              ...  
36565280.0        1
439564.0          1
35610100.0        1
10217873.0        1
1413000.0         1
Name: revenue, Length: 6863, dtype: int64

Por los valores observador arriba, podemos deducir que la columna a crear a continuación (return), va a ser mayormente 0.

In [216]:
data.loc[data["revenue"].isna(),"revenue"]=0

In [217]:
data.loc[data["budget"].isna(),"budget"]=0

In [218]:
data.dropna(subset=["release_date"],inplace=True)

In [219]:
# Quedan bastantes datos nulos, pero como en las transformaciones nos dicen de hacer nada más que esos puntos señalados
# No se tocan.
data.isna().sum()

belongs_to_collection    40888
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   2
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    249
spoken_languages             3
status                      83
tagline                  24981
title                        3
vote_average                 3
vote_count                   3
dtype: int64

In [220]:
data.shape

(45379, 18)

## Creo la columna "return"

In [221]:
# Verifico que sean de un tipo de dato divisible

print(type(data["revenue"][0]))
print(type(data["budget"][0]))
# La columna budget no puede simplemente pasarse a float ya que contiene valores que son string

<class 'numpy.float64'>
<class 'str'>


In [222]:
data["budget"]=pd.to_numeric(data["budget"],errors="coerce")
print(data["budget"].values)
print(data["budget"].unique())

[30000000. 65000000.        0. ...        0.        0.        0.]
[30000000. 65000000.        0. ...  3417000. 25868826.  1254040.]


In [223]:
data["budget"].value_counts()

0.0            36490
5000000.0        286
10000000.0       259
20000000.0       243
2000000.0        242
               ...  
270000000.0        1
923.0              1
72500000.0         1
2160000.0          1
1254040.0          1
Name: budget, Length: 1223, dtype: int64

In [246]:
# Ahora la idea es hacer la división pero con un try and except
# donde si la división va a tirar un error, sea porque budget es cero(Estará correcto este caso?) o string,
# se le deja el valor cero a la nueva columna.
columna=[]
for i, j in zip(data["revenue"], data["budget"]):
    if j!=0:
        try:
            columna.append(float(i)/(float(j)))
        except:
            columna.append(0)
    else:
        columna.append(0)

# Agrego la columna "revenue"
data["return"]=columna

In [247]:
data["return"].value_counts()

0.000000     39995
1.000000        20
2.000000        12
4.000000        11
5.000000         8
             ...  
17.000000        1
0.277950         1
5.411653         1
4.777778         1
0.282600         1
Name: return, Length: 5232, dtype: int64

## Creo la columna "release_year"
Va a tener los años en los que salió la pelicula, esta información la sacamos de la columna "release_date", a la cual justamente paso anteriormente a tipo datetipe, asi utilizando los valores de este tipo de datos se vuelve mucho más fácil el conseguir esta información.


In [225]:
data["release_date"] = pd.to_datetime(data["release_date"], format="%Y-%m-%d", errors="coerce")

In [227]:
print(type(data["release_date"][0]))
print(data["release_date"][0])
print((data["release_date"][0]).year)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
1995-10-30 00:00:00
1995


In [228]:
data["release_year"]=data["release_date"].dt.year

In [229]:
data[data["release_year"].isna()]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year
19730,0.065736,NaN,"[{'name': 'Carousel Productions', 'id': 11176}...",1997-08-20,104.0,Released,NaN,False,6.0,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,1.931659,NaN,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",2012-09-29,68.0,Released,NaN,False,7.0,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,2.185485,NaN,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",2014-01-01,82.0,Released,Beware Of Frost Bites,False,4.3,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
data[data["release_date"].isna()]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year
19730,0.065736,NaN,"[{'name': 'Carousel Productions', 'id': 11176}...",1997-08-20,104.0,Released,NaN,False,6.0,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,1.931659,NaN,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",2012-09-29,68.0,Released,NaN,False,7.0,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,2.185485,NaN,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",2014-01-01,82.0,Released,Beware Of Frost Bites,False,4.3,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [231]:
data[data["budget"].isna()]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year
19730,0.065736,NaN,"[{'name': 'Carousel Productions', 'id': 11176}...",1997-08-20,104.0,Released,NaN,False,6.0,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,1.931659,NaN,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",2012-09-29,68.0,Released,NaN,False,7.0,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,2.185485,NaN,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",2014-01-01,82.0,Released,Beware Of Frost Bites,False,4.3,NaT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [232]:
# Note gran coincidencia con respecto a la aparición de estos tres 
# en lo que viene a ser valores nulos, y parecen tener bastante errores
# al ser los tres únicos e inutilizables, tome la decisión de directamente quitarlos del dataframe

data.drop(labels=[19730,29503,35587],axis=0,inplace=True)

# Ahora me parece más limpio, y solo se quitaron tres lineas las cuales 
# no se podía distinguir sus datos, además de estar mayormente vacios, estaban intercambiados los valores
# (no sé exactamente de que forma se deberían reordenar)

In [233]:
data.isna().sum()

belongs_to_collection    40888
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    246
spoken_languages             0
status                      80
tagline                  24978
title                        0
vote_average                 0
vote_count                   0
return                       0
release_year                 0
dtype: int64

Acomodo las columas por gusto

In [235]:
# La forma en la que yo quiero ordenadas las columnas
data=data[['id', 'title','overview','status','runtime','popularity','belongs_to_collection','tagline','genres','original_language','spoken_languages','vote_average','vote_count','release_date','release_year','production_countries','production_companies','budget','revenue','return']]

In [253]:
data.sort_values(["return"], ascending=False).head()

,id,title,overview,status,runtime,popularity,belongs_to_collection,tagline,genres,original_language,spoken_languages,vote_average,vote_count,release_date,release_year,production_countries,production_companies,budget,revenue,return
4000,13703,Less Than Zero,A college freshman returns to Los Angeles for ...,Released,98.0,4.028679,NaN,"In Beverly Hills, you can have anything your h...","[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",6.1,77.0,1987-11-06,1987.0,"[{'iso_3166_1': 'US', 'name': 'United States o...",[{'name': 'Twentieth Century Fox Film Corporat...,1.0,12396383.0,1.239638e+07
3342,3082,Modern Times,The Tramp struggles to live in modern industri...,Released,87.0,8.159556,NaN,He stands alone as the greatest entertainer of...,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",en,"[{'iso_639_1': 'en', 'name': 'English'}]",8.1,881.0,1936-02-05,1936.0,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'name': 'United Artists', 'id': 60}, {'name'...",1.0,8500000.0,8.500000e+06
14372,14968,Welcome to Dongmakgol,"Based on the long running play by Jang Jin, th...",Released,133.0,4.222195,NaN,NaN,"[{'id': 10752, 'name': 'War'}, {'id': 35, 'nam...",ko,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",7.7,49.0,2005-08-04,2005.0,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]","[{'name': 'Film It Suda', 'id': 5659}]",8.0,33579813.0,4.197477e+06
22278,114903,Aquí Entre Nos,"Rodolfo Guerra, father of three daughters, wak...",Released,73.0,0.230075,NaN,Some marriages end well... others last a whole...,"[{'id': 35, 'name': 'Comedy'}]",en,"[{'iso_639_1': 'es', 'name': 'Español'}]",6.0,3.0,2012-03-30,2012.0,"[{'iso_3166_1': 'MX', 'name': 'Mexico'}]",[],1.0,2755584.0,2.755584e+06
2308,8856,"The Karate Kid, Part II",Mr. Miyagi and Daniel take a trip to Okinawa t...,Released,113.0,9.231318,"{'id': 8580, 'name': 'The Karate Kid Collectio...","This time, the combat is real.","[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",en,"[{'iso_639_1': 'en', 'name': 'English'}]",5.9,457.0,1986-06-18,1986.0,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'name': 'Columbia Pictures Corporation', 'id...",113.0,115103979.0,1.018619e+06


## Usar el otro archivo csv provisto
Voy a ver cómo estan los datos y de ahí intentar ponerlos de manera comoda para poder trabajar con ambos dataframe a la vez

In [236]:
df2=pd.read_csv(r"D:\Programacion\DataScience_Henry\Proyecto_Individual1\Dataset\credits.csv")

In [266]:
df2["crew"][0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [239]:
# Lo cambio al tipo de dato para que deje de detectarlo como objeto
# Asi puedo comparar los id de ambos dataframe
data["id"]=pd.to_numeric(data["id"],downcast="integer")

In [240]:
data[data["id"]==862]

,id,title,overview,status,runtime,popularity,belongs_to_collection,tagline,genres,original_language,spoken_languages,vote_average,vote_count,release_date,release_year,production_countries,production_companies,budget,revenue,return
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Released,81.0,21.946943,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"[{'iso_639_1': 'en', 'name': 'English'}]",7.7,5415.0,1995-10-30,1995.0,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'name': 'Pixar Animation Studios', 'id': 3}]",30000000.0,373554033.0,12.451801


In [241]:
import pandas as pd
import ast

# Función proveida por chatgpt para ayudar a desanidar
# Al ver cómo funcionan tengo que extraer los subdataframe que deja en la serie devuelta.

def desanidar_cast(row):
    cast_data = ast.literal_eval(row['cast'])
    desanidado = pd.json_normalize(cast_data)
    desanidado['id'] = row['id']
    return desanidado

# Repetir la misma función cambiando la variable -hay que ver si funciona-
def desanidar_crew(row):
    cast_data = ast.literal_eval(row['crew'])
    desanidado = pd.json_normalize(cast_data)
    desanidado['id'] = row['id']
    return desanidado

In [174]:
cast=df2.apply(desanidar_cast, axis=1).reset_index(drop=True)

In [242]:
crew=df2.apply(desanidar_crew, axis=1).reset_index(drop=True)

In [175]:
# Me devuelve una serie
# Pero si la hago dataframe, es como si quedara una columna en la que cada fila 
# Es un dataframe de cada uno de los datos que se querian separar de esa manera(desanidados)
print(cast)
print(type(cast))
pd.DataFrame(cast)

0            cast_id                character          ...
1            cast_id                       character   ...
2           cast_id                     character      ...
3           cast_id                  character         ...
4            cast_id              character            ...
                               ...                        
45471       cast_id character                 credit_id...
45472        cast_id              character            ...
45473        cast_id          character                ...
45474       cast_id character                 credit_id...
45475              Empty DataFrame
Columns: [id]
Index: []
Length: 45476, dtype: object
<class 'pandas.core.series.Series'>


,0
0,cast_id character ...
1,cast_id character ...
2,cast_id character ...
3,cast_id character ...
4,cast_id character ...
...,...
45471,cast_id character credit_id...
45472,cast_id character ...
45473,cast_id character ...
45474,cast_id character credit_id...


In [267]:
print(type(crew[0]))
crew[0]

<class 'pandas.core.frame.DataFrame'>


,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2,862,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2,862,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2,862,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2,862,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0,862,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...
101,589216c19251412dc2009cb9,Sound,1,862,Assistant Sound Editor,Susan Sanford,None
102,589216ccc3a3680973009274,Sound,0,862,Assistant Sound Editor,Susan Popovic,None
103,589216d79251412dc8009aa0,Sound,0,862,Assistant Sound Editor,Dan Engstrom,None
104,589216e49251412dcd009a4f,Production,1,862,Casting Consultant,Ruth Lambert,None


In [183]:
# Al verlo por separado, se nota cómo en cada fila quedaron en forma de dataframe
# Entonces lo que podría hacer es un ciclo que añada cada uno de los subdataframes
# A un solo dataframe mayor 
print(type(cast[0]))
cast[0]

<class 'pandas.core.frame.DataFrame'>


,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,862,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,862,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,862,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,862,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18,Rex (voice),52fe4284c3a36847f8024fa5,2,862,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
5,19,Hamm (voice),52fe4284c3a36847f8024fa9,2,862,John Ratzenberger,5,/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg
6,20,Bo Peep (voice),52fe4284c3a36847f8024fad,1,862,Annie Potts,6,/eryXT84RL41jHSJcMy4kS3u9y6w.jpg
7,26,Andy (voice),52fe4284c3a36847f8024fc1,0,862,John Morris,7,/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg
8,22,Sid (voice),52fe4284c3a36847f8024fb1,2,862,Erik von Detten,8,/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg
9,23,Mrs. Davis (voice),52fe4284c3a36847f8024fb5,1,862,Laurie Metcalf,9,/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg


In [184]:
# El último elemento no es un dataframe con datos, asi que no lo vamos a considerar
cast[45475]

,id


In [258]:
cast=list(cast)
df_cast=pd.concat(cast, ignore_index=True)

In [268]:
crew=list(crew)
df_crew=pd.concat(crew, ignore_index=True)

In [270]:
# Me aseguro que no hayan duplicados y elimino las columnas que considero innecesarias
df_crew.drop_duplicates()
df_crew.drop(columns=["credit_id","profile_path"],inplace=True)

In [269]:
# Elimino las columnas que considero innecesarias
df_cast.drop_duplicates()
df_cast.drop(columns=["credit_id","profile_path"],inplace=True)

In [ ]:
# Iba a usar esto, resulto en tardar más de 20 min, busque la manera de optimizarlo mientras veia que tardaba
# me canse, lo frene, y aplique la manera encontrada(la de arriba)
'''
for i in range(len(cast)-1):
    if i ==0:
        df_crew=cast[0]
    else:
        df_crew=pd.concat([df_crew,cast[i]],ignore_index=True)
'''

In [275]:
# Queria verificar que cast_id sirviera como columna id, pero justamente no lo hace.
# Por lo tanto tal vez tenga otros usos para las personas originales de los datos
# Pero para busquedas y comparaciones no se pueden usar.
# (otro caso es el id, que si bien se repite, donde esta el original y no se debería repetir es en la tabla de datos, que seria la tabla dimensional)

df_cast["cast_id"].value_counts()

4.0       26777
5.0       25655
3.0       25594
6.0       24006
2.0       22846
          ...  
344.0         1
1099.0        1
1098.0        1
1097.0        1
438.0         1
Name: cast_id, Length: 574, dtype: int64

In [271]:
df_crew

,department,gender,id,job,name
0,Directing,2.0,862,Director,John Lasseter
1,Writing,2.0,862,Screenplay,Joss Whedon
2,Writing,2.0,862,Screenplay,Andrew Stanton
3,Writing,2.0,862,Screenplay,Joel Cohen
4,Writing,0.0,862,Screenplay,Alec Sokolow
...,...,...,...,...,...
464309,Sound,0.0,67758,Original Music Composer,Richard McHugh
464310,Camera,2.0,67758,Director of Photography,João Fernandes
464311,Directing,0.0,227506,Director,Yakov Protazanov
464312,Production,2.0,227506,Producer,Joseph N. Ermolieff


In [133]:
# Como todavía no sé cómo se va a trabajar la parte de machine learning
# Pero esta limpieza de datos me parece necesaria
# Voy a crear una copia del dataframe tal como esta ahora
# (Todavía falta desanidar los datos, cuando lo haga veo si lo vuelvo a copiar)

data2= data.copy()
# Más adelante decido si sirve o no.

In [280]:
data[data["id"]==141971]

,id,title,overview,status,runtime,popularity,belongs_to_collection,tagline,genres,original_language,spoken_languages,vote_average,vote_count,release_date,release_year,production_countries,production_companies,budget,revenue,return
13261,141971,Blackout,Recovering from a nail gun shot to the head an...,Released,108.0,0.411949,NaN,Which one is the first to return - memory or t...,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",fi,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",6.7,3.0,2008-12-26,2008.0,"[{'iso_3166_1': 'FI', 'name': 'Finland'}]","[{'name': 'Filmiteollisuus Fine', 'id': 5166}]",0.0,0.0,0.0
13375,141971,Blackout,Recovering from a nail gun shot to the head an...,Released,108.0,0.411949,NaN,Which one is the first to return - memory or t...,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",fi,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",6.7,3.0,2008-12-26,2008.0,"[{'iso_3166_1': 'FI', 'name': 'Finland'}]","[{'name': 'Filmiteollisuus Fine', 'id': 5166}]",0.0,0.0,0.0
16764,141971,Blackout,Recovering from a nail gun shot to the head an...,Released,108.0,0.411949,NaN,Which one is the first to return - memory or t...,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",fi,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",6.7,3.0,2008-12-26,2008.0,"[{'iso_3166_1': 'FI', 'name': 'Finland'}]","[{'name': 'Filmiteollisuus Fine', 'id': 5166}]",0.0,0.0,0.0


In [277]:
data["id"].value_counts()

141971    3
97995     2
10991     2
109962    2
119916    2
         ..
40594     1
23104     1
37605     1
91727     1
461257    1
Name: id, Length: 45346, dtype: int64

## Es hora de desanidar los datos
No considero necesario desanidarlos, pero tampoco sé qué necesito y qué no, asi que decidí intentar ir primero por preparar las funciones de API (bastante a ojo e irlas mejorando conforme cambiaba lo que creía necesario en esta categoría).

Quiero intentar quedarme con la columna de generos sin desanidar, pero eso depende de cómo esté escrita esta columna, y si todas están escritas de igual manera.

In [83]:
print(data["genres"][0])
print(type(data["genres"][0]))
# Viene en forma de diccionario. 
# Asi que para mi idea es aprender a sacar solamente los generos en la función o 
# Tomar los generos y meterlos a una lista.

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
<class 'str'>


In [84]:
data["genres"]=dict(data["genres"])

In [85]:
print(data["genres"][0])
print(type(data["genres"][0]))

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
<class 'str'>


# Funciones para la API

def peliculas_idioma( Idioma: str ): Se ingresa un idioma. Debe devolver la cantidad de películas producidas en ese idioma.<br>
 Ejemplo de retorno: X cantidad de películas fueron estrenadas en idioma

def peliculas_duracion( Pelicula: str ): Se ingresa una pelicula. Debe devolver la duracion y el año.<br>
Ejemplo de retorno: X . Duración: x. Año: xx

def franquicia( Franquicia: str ): Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio.<br>
Ejemplo de retorno: La franquicia X posee X peliculas, una ganancia total de x y una ganancia promedio de xx

def peliculas_pais( Pais: str ): Se ingresa un país, retornando la cantidad de peliculas producidas en el mismo.<br>
Ejemplo de retorno: Se produjeron X películas en el país X

def productoras_exitosas( Productora: str ): Se ingresa la productora, entregandote el revunue total y la cantidad de peliculas que realizo.<br>
Ejemplo de retorno: La productora X ha tenido un revenue de x

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma, en formato lista.